# canvod-aux

GNSS auxiliary data (ephemerides and clock corrections)

## Overview

`canvod-aux` provides tools for downloading, parsing, and managing GNSS auxiliary data:

- **SP3 Ephemerides**: Precise satellite orbit data
- **CLK Corrections**: Satellite clock bias corrections
- **Multi-source Support**: NASA, ESA, CODE, GFZ, and more
- **Automatic Selection**: Intelligent fallback between data centers
- **Interpolation**: High-precision orbital state computation

## Quick Start

### Installation

```bash
pip install canvod-aux
```

### Basic Usage

#### 1. Download SP3 Ephemerides

In [ ]:
from canvod.aux import SP3Container, registry
from canvod.aux._internal import YYYYDOY

# Create container with CODE rapid product
container = SP3Container(
    yyyydoy=YYYYDOY.from_date_args(2024, 1, 1),
    sp3_type="cod_rap"
)

# Download and parse
sp3_data = container.download_and_parse()
print(f"Loaded {len(sp3_data.satellites)} satellites")
print(f"Time span: {sp3_data.start_epoch} to {sp3_data.end_epoch}")

#### 2. Download Clock Corrections

In [ ]:
from canvod.aux import CLKContainer

# Create container with matching product type
clk_container = CLKContainer(
    yyyydoy=YYYYDOY.from_date_args(2024, 1, 1),
    clk_type="cod_rap"
)

# Download and parse
clk_data = clk_container.download_and_parse()
print(f"Loaded clock data for {len(clk_data.satellites)} satellites")

#### 3. Interpolate Satellite States

In [ ]:
from datetime import datetime, timezone

# Interpolate position for specific satellite and time
epoch = datetime(2024, 1, 1, 12, 0, 0, tzinfo=timezone.utc)
sat_id = "G01"

position = sp3_data.interpolate_position(sat_id, epoch)
print(f"Position at {epoch}:")
print(f"  X: {position.x:.3f} km")
print(f"  Y: {position.y:.3f} km")
print(f"  Z: {position.z:.3f} km")

## Core Components

### Product Registry

Centralized configuration for all auxiliary data products:

In [ ]:
from canvod.aux import registry

# List all available products
print("Available SP3 products:")
for key in registry.list_sp3_products():
    product = registry.get_sp3_product(key)
    print(f"  {key}: {product.name} ({product.source})")

print("\nAvailable CLK products:")
for key in registry.list_clk_products():
    product = registry.get_clk_product(key)
    print(f"  {key}: {product.name} ({product.source})")

### Data Containers

Containers handle download, caching, and parsing:

- **SP3Container**: Manages ephemeris files
- **CLKContainer**: Manages clock correction files

Both provide:
- Automatic server selection (primary/backup)
- Local caching
- Compression handling (`.Z`, `.gz`)
- Validation

## Advanced Features

### Custom Product Configuration

In [ ]:
from canvod.aux import SP3Container
from canvod.aux._internal import YYYYDOY

# Override default download directory
container = SP3Container(
    yyyydoy=YYYYDOY.from_date_args(2024, 1, 1),
    sp3_type="cod_rap",
    sp3_dir="/custom/path/to/sp3"
)

# Force redownload
sp3_data = container.download_and_parse(redownload=True)

### Multi-Product Workflow

In [ ]:
from canvod.aux import SP3Container, CLKContainer
from canvod.aux._internal import YYYYDOY

# Process multiple days
dates = [YYYYDOY.from_date_args(2024, 1, day) for day in range(1, 8)]

for yyyydoy in dates:
    # Download ephemerides and clocks
    sp3 = SP3Container(yyyydoy, "cod_rap").download_and_parse()
    clk = CLKContainer(yyyydoy, "cod_rap").download_and_parse()
    
    print(f"{yyyydoy}: {len(sp3.satellites)} satellites")

### Interpolation Options

In [ ]:
from datetime import datetime, timezone

# Different interpolation methods
epoch = datetime(2024, 1, 1, 12, 0, 0, tzinfo=timezone.utc)

# Lagrange polynomial (default, order=10)
pos_lagrange = sp3_data.interpolate_position("G01", epoch, method="lagrange", order=10)

# Cubic spline
pos_spline = sp3_data.interpolate_position("G01", epoch, method="spline")

# Include velocity
pos, vel = sp3_data.interpolate_state("G01", epoch)

## Data Sources

Supported analysis centers:

| Center | Products | Latency | Accuracy |
|--------|----------|---------|----------|
| **CODE** | Final, Rapid | 11d / 17h | <2.5cm / <5cm |
| **ESA** | Final, Rapid | 13d / 2d | <2.5cm / <5cm |
| **GFZ** | Final, Rapid | 11d / 17h | <2.5cm / <5cm |
| **NASA** | Final, Rapid, Ultra | 13d / 17h / 3h | <2.5cm / <5cm / <5cm |

### Product Keys

Products use standardized naming:
- `cod_fin`: CODE Final
- `cod_rap`: CODE Rapid
- `esa_fin`: ESA Final
- `esa_rap`: ESA Rapid
- `gfz_fin`: GFZ Final
- `gfz_rap`: GFZ Rapid
- `nas_fin`: NASA Final
- `nas_rap`: NASA Rapid
- `nas_ult`: NASA Ultra-Rapid

## API Reference

### Main Classes

```python
class SP3Container:
    """Manages SP3 ephemeris file lifecycle."""
    
    def __init__(self, yyyydoy: YYYYDOY, sp3_type: str, sp3_dir: Path | None = None)
    def download_and_parse(self, redownload: bool = False) -> SP3Data

class CLKContainer:
    """Manages CLK clock correction file lifecycle."""
    
    def __init__(self, yyyydoy: YYYYDOY, clk_type: str, clk_dir: Path | None = None)
    def download_and_parse(self, redownload: bool = False) -> CLKData

class SP3Data:
    """Parsed SP3 ephemeris data."""
    
    satellites: list[str]
    start_epoch: datetime
    end_epoch: datetime
    
    def interpolate_position(self, sat_id: str, epoch: datetime, **kwargs) -> Position
    def interpolate_state(self, sat_id: str, epoch: datetime, **kwargs) -> tuple[Position, Velocity]

class CLKData:
    """Parsed CLK clock correction data."""
    
    satellites: list[str]
    
    def get_clock_bias(self, sat_id: str, epoch: datetime) -> float
```

## Testing

Run tests:

```bash
just test
```

With coverage:

```bash
just test-cov
```